In [5]:
import pandas as pd


from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build

In [6]:
year_folder_id_dict = {'2010':'113DAB_g6P3b-gbrpFog682EfdILcBMt6','2011':'1Yh9610KUMxZLNDiyUOnng-7PbhlYbt3x',
                       '2012':'1TEWAyyxf6dJ3K6hMiliayJ4VV4ilvwIH','2013':'1KWNg6Me11spV-JRFOzwlvNpvoYMLI59C',
                       '2014':'1IgDuaPlc9b-MdP6VELFXcTffqSRHYPU5','2015':'1CrHueAEH76p95tIOryMrdu5nV-qW702M',
                       '2016':'1hpViY56I20A6k6EeQGrOn6QG11j7gb58','2017':'1H2HXwALMyzxBIMJyluYkTHJKplqGIpUF',
                       '2018':'1-jlTY-u_izedKKNenevC3kgM6yaKR7b7','2019':'1Qm-Sse4vbOx2IOAILUCTvXwqShWlynQn'}

events_folder_id_dict = {'2010':'1CPmsZoNqsDslYUDuMMYAiTRdgcbjuIj6','2011':'1FQenIULvAsgQN4sLSpcLdjjfStfeIvga',
                       '2012':'1ABi7lAAmlARbrGlBJv8Tz6IFXVSYzLdR','2013':'1MMg8bUfZj3eZf_gAlr3k2N9_LWmO5DWr',
                       '2014':'135rCpr_1_N00nmsKOcyWjmQPmT_n8I96','2015':'1Gjm0qtTXL-21qNQJtloxNGtJq3Q1Xlyq',
                       '2016':'1qRUeOe4mFx5ofuYHxiqL4u1HvqxLCz96','2017':'15DY5sNvP8O2jgus7Qythy8Nzeeo7cZAI',
                       '2018':'1AKHSz-t3vkUsLE2IC4TdrMUznjNC3YpU','2019':'1qOX3DQFtU_j4czyUMndjpn9c0crZ8ue1'}
events_folder_id = '165-gELn98fyKGaWnmWSkJuUuz1aj8XMg'

In [7]:
df = pd.read_csv('df_results.csv', index_col=0)

In [11]:
creds = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-92c3baf53205.json',
                                                         scopes=['https://www.googleapis.com/auth/drive'])
service = build('drive', 'v3', credentials=creds)

In [18]:
saved_files = []
for folder_id in year_folder_id_dict.values():
    results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)").execute()
    files = results.get("files", [])
    next_page_token = results.get("nextPageToken")
    while next_page_token:
        results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)", pageToken=next_page_token).execute()
        files.extend(results.get("files", []))
        next_page_token = results.get("nextPageToken")
    for file in files:
        saved_files.append(file.get('name').replace('.pdf',''))
print(len(saved_files))

1734


In [20]:
mask = df['date'].isin(saved_files)
df.loc[mask, 'GD'] = 'raw'

In [23]:
mask = ~df['date'].isin(saved_files)
df.loc[mask, 'GD'] = 'missed'

In [26]:
df['GD'].value_counts()

raw       1735
missed    1193
Name: GD, dtype: int64

In [30]:
with open('missing_dates.txt', 'r') as f:
    missing_dates = [x.strip() for x in f.read().split('\n')]

In [43]:
mask = (~df['date'].isin(missing_dates)) & (df['GD']=='missed')
df_missing_files = df.loc[mask]
df_missing_files.to_csv('df_missing_files.csv')